---
# **Title**:TRABAJO AP. DDPG TD(3) (Gradiente de política determinista profunda (TD3) de doble retardo. Entornos de estados continuos: Humanoide). + Conexión Chatgpt (o similar)  + modelo dalle (similr)
# **Author**: José Javier Gutiérrez Gil
# **Date**: 2024-02-18
# ***Univeridad de Valencia. Grado de Ciencia de Datos***
---

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/td3')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/td3')

In [ ]:
!pip install pybullet
!pip install gym==0.22.0 # Versión más actual que contiene la calse Monitor y así poder crear los videos del entrenamiento. Al monos en la 0.23 me da error

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.22.0-py3-none-any.whl size=708364 sha256=894a34ab0091e5ac006564e0162cd94f6e1028c4d5086cd234f4f6a2ad943026
  Stored in directory: /root/.cache/pip/wheels/42/e8/e8/6dfbc92a1dcd76c1a5e2bb982750fd6b7e792239f46039e6b1
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


# **Step 0:** Cargamos las funciones necesarias de nuestra libreria y de python

In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#####
import gym
import pybullet_envs
from gym import wrappers
##
import torch
import torch.nn as nn
#####
from TD3 import TD3, ReplayBuffer
from TD3 import created_models_directory, mkdir
from TD3 import evaluate_train_policy, evaluate_policy
from TD3 import noisy_action_wrapper, save_env, load_env, create__metrics_imagen
from TD3 import serialize_object, lists_to_serializable_object, serialize_training


# **Step 1:** Inicializamos los hiperparámetros del modelo e implementación

In [ ]:

env_name            =  "HumanoidBulletEnv-v0"  # Nombre del entorno que vamos a entrena. Con forme se ha implementado el modelo TD3 poría ser cuaalquier entorno (preferiblemente de estados de acciones continuos)
seed                = 0        # Semilla utilizada para garantizar la reproducibilidad de los resultados
initial_memory_prob = 0.1      # Define la probabilidad inicial de tomar una acción de la memoria de repetición
max_start_steps     = 1e4      # Define el número máximo de pasos iniciales
#start_steps         = 1e4#1e4  --123-- lo modificamos por un decaimiento según los pasos de entrenamiento # Número de iteraciones/timesteps antes de que el modelo comience a utilizar la red de políticas en lugar de elegir acciones al azar
eval_frequency      = 5e3      # Frecuencia de evaluación, es decir, cada cuántos pasos/timesteps se evalúa el desempeño del modelo
max_timesteps       = 1e6      # Número máximo de iteraciones/timesteps permitidos
save_models         = True     # Booleano que indica si se deben guardar los modelos pre-entrenados o no
max_explore_noise   = 0.1      # Desviación estándar del ruido gaussiano utilizado para la exploración...--123-- CaMBIO  0.01 por 0.2 y creo un current_noise_explore dependiendo del steo en el que estamos
batch_size          = 100      # Tamaño del lote de muestras utilizadas en cada iteración de entrenamiento
gamma               = 0.99     # Factor de descuento gamma que afecta la importancia de las recompensas futuras en la función de pérdida
target_update_freq  = 0.005    # Tasa de actualización para suavizar los parámetros de la red objetivo
policy_noise        = 0.2      # Desviación estándar del ruido gaussiano agregado a las acciones para promover la exploración
noise_clip          = 0.5      # Valor máximo permitido para el ruido gaussiano agregado a las acciones (política)
policy_freq         = 2        # Número de iteraciones entre actualizaciones de la red de políticas (modelo actor)

# **Step 2:** Creamos los directorios y cargamos el entorno de trabajo

In [ ]:
#Creamos los directorios de almacenamento de resultados y métricas
file_model_name = created_models_directory (env_name, seed, save_models, "v2_1")

# Cargamos el entorno sobre el cual ejecutaremso el modelo DDPG TD3
###############################################################################
##  CARGAMOS EL ENTORNO HUMANOIDE V0 Y LE INTRODUCIMOS EL WRAPPER PARA AÑADIR RUIDO A
##  LAS ACCIONES OBTENIDAS POR EXPLORACIÓN.
################################################################################
env = gym.make (env_name)
max_episode_steps = env._max_episode_steps
env = noisy_action_wrapper (env, noise_level = policy_noise) #le agregamos ruido a la accion obtenida del entorno para darle ms estabilidad al entrenamiento
# Fijamos la semilla y obtenemos información del entorno (Estados, acciones)
env.seed (seed)
torch.manual_seed (seed)
np.random.seed (seed)

state_dim  = env.observation_space.shape [0]
action_dim = env.action_space.shape [0]
max_action = float(env.action_space.high [0])

#Creamos los directorios de trabajo donde guardará los videos del entrenamiento
work_dir          = mkdir ('exp', 'brs')
monitor_dir       = mkdir (work_dir, 'monitor')

save_env_vid      = False
if save_env_vid:
  env = wrappers.Monitor (env, monitor_dir, force = True)
  env.reset()


---------------------------------------
Fichero de los modelos entrenados: TD3_HumanoidBulletEnv-v0_0_v2_1
---------------------------------------


# **Step 3:** Proceso de entrenamiento : e.gredy



**Estrategia ε-greedy adaptativa:**

Durante el entrenamiento, utilizamos una estrategia ε-greedy para equilibrar la exploración y la explotación.
- En la fase de exploración inicial (cuando total_steps < max_start_steps), seleccionamos acciones al azar con una probabilidad alta de exploración (ε), lo que nos permite recopilar información sobre el entorno.
- A medida que el agente acumula más experiencia, disminuimos gradualmente la probabilidad de exploración (ε) utilizando una función de decaimiento lineal, pasando de ε_start a ε_end en ε_decay_steps pasos.
- En la fase de explotación (cuando total_steps >= max_start_steps), seleccionamos acciones principalmente explotando las estimaciones actuales de los valores Q. Sin embargo, aún mantenemos la capacidad de explorar introduciendo un pequeño nivel de ruido en las acciones seleccionadas, controlado por max_explore_nois. Este ruido garantiza que el agente continúe explorando incluso durante la fase de explotación, aunque en menor medida que durante la fase inicial de exploración.
- Al final de cada episodio, actualizamos el valor de ε para garantizar que disminuya con el tiempo, lo que refleja la idea de que a medida que el agente adquiere más experiencia, confiamos menos en la exploración aleatoria y más en la explotación de las estimaciones aprendidas.

- Este enfoque nos permite equilibrar eficazmente la exploración y la explotación durante el entrenamiento, lo que facilita el descubrimiento de políticas óptimas en entornos complejos.

In [ ]:
class EGreedyPolicy:
    '''
        - __init__: Este método inicializa los parámetros de la política ε-greedy, como
          los valores iniciales y finales de ε, y el número de pasos de decaimiento de ε.

        - select_action: Este método elige una acción de acuerdo con la estrategia ε-greedy.
        Si un número aleatorio es menor que ε, se selecciona una acción al azar; de lo contrario,
        se selecciona la mejor acción según los valores Q.

        - update_epsilon: Este método actualiza el valor de ε durante el entrenamiento. Disminuye
        linealmente desde el valor inicial hasta el valor final a lo largo del número
        especificado de pasos de decaimiento
    '''
    def __init__(self, env, policy, epsilon_start, epsilon_end, epsilon_decay_steps):
        self.env    = env
        self.policy = policy
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay_steps = epsilon_decay_steps
        self.epsilon = epsilon_start

    def select_action(self, obs):
        if np.random.rand() < self.epsilon:
            print ("Exploracion 2")
            return self.env.action_space.sample ()
        else:
            print ("Explotacion")
            return self.policy.select_action (np.array (obs))

    def update_epsilon(self, step):
        # Disminuye epsilon linealmente desde epsilon_start hasta epsilon_end
        if step < self.epsilon_decay_steps:
            self.epsilon = self.epsilon_start - (step / self.epsilon_decay_steps) * (self.epsilon_start - self.epsilon_end)
        else:
            self.epsilon = self.epsilon_end

class AdaptiveEGreedyPolicy:
    def __init__(self, env, policy, epsilon_start, epsilon_end, epsilon_decay_steps, max_timesteps):
        self.env    = env
        self.policy = policy
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay_steps = epsilon_decay_steps
        self.max_timesteps = max_timesteps
        self.epsilon = epsilon_start

    def select_action(self, total_steps, obs):
        if np.random.rand() < self.epsilon:
            #print ("Exploracion 2")
            return self.env.action_space.sample()
        else:
            #print ("Explotacion 1")
            # Acceder a la experiencai del agente
            action = self.policy.select_action (np.array (obs))
            # print('Acción de la experiencia del agente:', action)
            current_explore_noise = max_explore_noise * max(1 - total_steps / max_timesteps, 0)
            if current_explore_noise != 0:
                action = (action + np.random.normal(0, max_explore_noise, size=action_dim)).clip(env.action_space.low, env.action_space.high)
            # print('Acción seleccionada por el modelo:', action)
        return action

    def update_epsilon(self, total_steps):
        if total_steps < self.epsilon_decay_steps:
            self.epsilon = self.epsilon_start - (total_steps / self.epsilon_decay_steps) * (self.epsilon_start - self.epsilon_end)
        else:
            self.epsilon = self.epsilon_end

# Creamos el objeto DDPG TD3 con los hiperparámetros definidos anteriormente
policy = TD3 (state_dim, action_dim, max_action,max_timesteps, initial_lr =  1e-4) # Notar que realmente el objeto entrenado definirá la política a seguir por el agente

# Creamos un objeto EGreedyPolicy para la exploración
epsilon_start       = 1.0
epsilon_end         = 0.1
epsilon_decay_steps = 1000
exploration_policy  = AdaptiveEGreedyPolicy (env, policy, epsilon_start, epsilon_end, epsilon_decay_steps,max_timesteps) #EGreedyPolicy(epsilon_start, epsilon_end, epsilon_decay_steps)

# Nos creamos la memoria de repetición de experiencias
replay_buff = ReplayBuffer ( max_capacity = 2e4) # Notar que la máxima capacidad de la memoria por defecto es: max_capacity = 1e4
# replay_buff = SimplePrioritizedReplayBuffer()

# Lista donde se guardarán las evaluaciones de la política durante el entrenamiento
evaluations = [evaluate_train_policy (policy, env)]

# Inicialización de variables utilizadas en el entrenamiento
total_steps        = 0     # Número total de pasos de entrenamiento que llevamos realizados
episode_steps      = 0     # Número de pasos realizados en el epidodeo
steps_since_eval   = 0     # Número de pasos desde la última evaluación de la política
episode_num        = 0     # Número de episodios completados durante el entrenamiento
done               = True  # Indica si el episodio actual ha finalizado, nos lo devuelve el entorno de gym

t0 = time.time()  # Tiempo inicial de referencia para medir el tiempo de entrenamiento
n_steps_epochs = []
all_rewards    = []
avg_losses_c   = []
avg_losses_a   = []
all_losses_c   = []
all_losses_a   = []
target_qs_c1   = []
target_qs_c2   = []
target_qs      = []
all_exploration_factor = []
############################################################################
# step 0: Bucle principal del entrenameinto de nuestro modelo.
##  REalizamos tantos pasos como indica el hiperparámetro "max_timesteps"
## A no ser que se estanque en un maximo local (mínimo) el humanoide aprenderá
## mejor cuantos mñas pasos realiza. Eta implementacición V2 utiliza una estrategia e-grady
## cuando finaliza el periodo de exploración. Es un entorno muy complicado con
## muchos grados de libertad y acciones infinitas por lo que es facil estancarse.
## Por ello, es muy importante el ruido aladido a las acciones.
#############################################################################

while total_steps < max_timesteps: # son pasos de tiempo
    # step 1: Se comprueba si el episodio ha concluido. Si lo ha hehco, realiamso el entrenamiento del modelo
    # con las acciones almacenadas en ReplayBuffer
    if done:
        # Comienza el entrenamiento del modelo si no es la primera iteración
        if total_steps != 0: #Ya que no se tendrá nada en la memoria de repetición
            print (f'Número de pasos del episodio {episode_num} son episode_steps:{episode_steps}')
            n_steps_epochs.append (episode_steps)
            # Realizamos el entrenamiento del modelo con las aciones almacenadas en ReplayBuffer
            rewards, losses_c,losses_a, target_qs_critic1, target_qs_critic2, target_qs = policy.train (replay_buff, episode_steps, batch_size, gamma,
                                                                               target_update_freq, policy_noise, noise_clip, policy_freq)
            #############
            # Nos guardamos las métricas del entrenamiento a estudio
            #############
            avg_reward = np.mean (rewards)
            avg_loss_c = np.mean (losses_c)
            avg_loss_a = np.mean (losses_a)
            # avg_expl_fact = np.mean(expl_fact)

            all_rewards.append (avg_reward)
            avg_losses_c.append (avg_loss_c)
            avg_losses_a.append (avg_loss_a)
            all_losses_c.append (losses_c.copy())
            all_losses_a.append (losses_a.copy())

            target_qs_c1.append (target_qs_critic1.copy())
            target_qs_c2.append (target_qs_critic2.copy())
            target_qs.append (target_qs.copy())

            # all_exploration_factor.append(avg_expl_fact)

            print("Total Steps: {} Episode Num: {} Reward: {} avg_loss_c: {} avg_loss_a: {}".format(total_steps, episode_num, episode_reward, avg_loss_c, avg_loss_a))


        # step 2: Se evalúa el rendimiento del episodio actual y se guarda la política
        # si se cumplen ciertas condiciones o criterios predefinidos.
        if steps_since_eval >= eval_frequency:
            steps_since_eval %= eval_frequency
            evaluations.append (evaluate_train_policy (policy, env))
            # Almacenamos el modelo en el estado de entrenamiento en el que se encuentra
            policy.save(file_model_name, directory="./pytorch_models")

            np.save("./results/%s" % (file_model_name), evaluations)

            save_env(file_model_name, directory="./results", env=env) # Guardamos el entorno en el estado actual
            # Serializar el ReplayBufferMemory y guardarlo en un archivo
            serialize_object(replay_buff, './results/replay_buffer_memory_v2_1.pickle')
            # Serializamos las métricas del entrenamiento
            lists_train_metrics = [all_rewards, avg_losses_c, avg_losses_a, all_losses_c, all_losses_a]
            attribute_names = ['rewards', 'losses']
            lists_to_serializable_object(lists_train_metrics, attribute_names, './results/serialized_list_train_metrics_v2_1.pickle')


        # step3: Reiniciar el entorno cuando finaliza el episodio de entrenamiento
        # Notar que según la logica del bucle, en el primer ciclo, este serña el primer paso.
        obs = env.reset()

        # Establecer "done" a Falso para parar el episodio
        done = False

        # Restablecer la recompensa del episodio y el contador de pasos del episodio
        episode_reward = 0
        episode_steps  = 0
        episode_num   += 1

    # step 4: Tomar acciones aleatorias antes de alcanzar el número de pasos iniciales
    if total_steps < max_start_steps:
        #print ("Exploracion 1")
        # Si estamos en los primeros pasos, acceder al entorno real (exploración) con una probabilidad inversamente proporcional
        # a la probabilidad de tomar una acción de la experiencia del agente
        action = env.action_space.sample()
     # Después de superar los primeros pasos, usar el modelo entrenado con una probabilidad inversamente proporcional
    # a la probabilidad de tomar una acción de la experiencia
    else:
      # Seleccionar la acción según la política ε-greedy
      exploration_policy.update_epsilon (total_steps)
      action = exploration_policy.select_action (total_steps, np.array (obs))


    # step 5: El agente ejecuta una acción en el entorno, lo que resulta en una transición
    # de estado. Además, el agente recibe una recompensa del entorno como resultado
    # de su acción. Esta acción puede cambiar el estado del entorno y, por lo tanto,
    # influir en las futuras observaciones y recompensas del agente.
    #print (f' * nueva action:  -- tamaño:{len (action)}')
    new_obs, reward, done, _ = env.step (action)
    #print (f' * nuevo reward:{reward}')
    # Comprueba si el episodio ha terminado
    done_bool = 0 if episode_steps + 1 == max_episode_steps else float(done)

    # Aumenta la recompensa total del episodio
    episode_reward += reward

    # step 6: Almacenar nueva transición en el búfer de repetición de experiencias
    ## Hemos desestimado el buffer de repeticion con prioridad por el costo computacional
    replay_buff.add((obs, new_obs, action, reward, done_bool))

    #error_explora = 0.01 # añadimos un error muy bajo para asegurarnos que al menos se utiliza una vez
    #replay_buff.add(error_explora, (obs, new_obs, action, reward, done_bool))

    # Actualizar estado, tiempo de paso del episodio, tiempo total de pasos y pasos desde la última evaluación de la política
    obs = new_obs
    episode_steps    += 1
    total_steps      += 1
    steps_since_eval += 1

    # Llamamos a la funcion para crear graficas de entrenamiento
    create__metrics_imagen (evaluations, all_rewards,  avg_losses_c,avg_losses_a, all_losses_c,
                                                    all_losses_a, target_qs_c1,
                                                    target_qs_c2, target_qs,
                                                    n_steps_epochs, "v2_1", episode_num, total_steps)

tf = time.time()  # Tiempo final de referencia para medir el tiempo de entrenamiento
serialize_training (t0, tf, total_steps,"v2")
# Añadir la última actualización de la política a la lista de evaluaciones previa y guardar nuestro modelo
evaluations.append (evaluate_train_policy (policy, env))
if save_models:
    policy.save("gready_%s" % (file_model_name), directory="./pytorch_models")
np.save("./results/gready_%s" % (file_model_name), evaluations)



Se han truncado las últimas 5000 líneas del flujo de salida.
Número de pasos del episodio 12030 son episode_steps:193
Total Steps: 753892 Episode Num: 12030 Reward: 155.2293385380441 avg_loss_c: 18.88262728952991 avg_loss_a: -57.12634623235989
Número de pasos del episodio 12031 son episode_steps:122
Total Steps: 754014 Episode Num: 12031 Reward: 66.72290640438632 avg_loss_c: 19.09537341164761 avg_loss_a: -57.60784780783732
Número de pasos del episodio 12032 son episode_steps:135
Total Steps: 754149 Episode Num: 12032 Reward: 48.32532222263281 avg_loss_c: 18.186251845183197 avg_loss_a: -56.42457131279839
Número de pasos del episodio 12033 son episode_steps:124
Total Steps: 754273 Episode Num: 12033 Reward: 108.08807126337342 avg_loss_c: 18.86636011831222 avg_loss_a: -56.962593201668035
Número de pasos del episodio 12034 son episode_steps:72
Total Steps: 754345 Episode Num: 12034 Reward: -20.37904941982382 avg_loss_c: 19.341667188538445 avg_loss_a: -56.636052237616646
Número de pasos del

AttributeError: 'list' object has no attribute 'appen'

# **Step 4:** Evaluación para extraeer videos de la politicca entrenada

In [ ]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque
#import mujoco_py
from TD3 import TD3, evaluate_policy, created_models_directory
import pybullet as p

# Iniciar el servidor de física de PyBullet
p.connect(p.DIRECT)



#######################################
###
######################################
env_name           = "HumanoidBulletEnv-v0"  # Nombre del entorno que vamos a entrena. Con forme se ha implementado el modelo TD3 poría ser cuaalquier entorno (preferiblemente de estados de acciones continuos)
seed               = 0        # Semilla utilizada para garantizar la reproducibilidad de los resultados
start_steps        = 1e4      # Número de iteraciones/timesteps antes de que el modelo comience a utilizar la red de políticas en lugar de elegir acciones al azar
eval_frequency     = 5e3      # Frecuencia de evaluación, es decir, cada cuántos pasos/timesteps se evalúa el desempeño del modelo
max_timesteps      = 5e12     # Número máximo de iteraciones/timesteps permitidos
save_models        = True     # Booleano que indica si se deben guardar los modelos pre-entrenados o no
explore_noise      = 0.1      # Desviación estándar del ruido gaussiano utilizado para la exploración
batch_size         = 100      # Tamaño del lote de muestras utilizadas en cada iteración de entrenamiento
gamma              = 0.99     # Factor de descuento gamma que afecta la importancia de las recompensas futuras en la función de pérdida
target_update_freq = 0.005    # Tasa de actualización para suavizar los parámetros de la red objetivo
policy_noise       = 0.2      # Desviación estándar del ruido gaussiano agregado a las acciones para promover la exploración
noise_clip         = 0.5      # Valor máximo permitido para el ruido gaussiano agregado a las acciones (política)
policy_freq        = 2        # Número de iteraciones entre actualizaciones de la red de políticas (modelo actor)

env_name = "HumanoidBulletEnv-v0"
seed = 0

work_dir = os.path.join('exp', 'brs')
monitor_dir = os.path.join(work_dir, 'monitor')


file_model_name = created_models_directory (env_name, seed, save_models, "v2_1")
print ("---------------------------------------")
print ("Configuración: %s" % (file_model_name))
print ("---------------------------------------")

eval_episodes = 1000000
save_env_vid  = True

env = gym.make (env_name)
max_episode_steps = env._max_episode_steps
if save_env_vid:
  env = wrappers.Monitor (env, monitor_dir, force = True)
  # env = RecordEpisodeStatistics (env)
  env.reset ()
env.seed (seed)

torch.manual_seed (seed)
np.random.seed (seed)

state_dim  = env.observation_space.shape [0]
action_dim = env.action_space.shape [0]
max_action = float (env.action_space.high [0])

policy = TD3 (state_dim, action_dim, max_action,max_timesteps, initial_lr =  1e-4) # Notar que realmente el objeto entrenado definirá la política a seguir por el agente
policy.load (file_model_name, './pytorch_models/')
avg_reward, episode_rewards = evaluate_policy (policy, env, eval_episodes = eval_episodes)

# Graficar las recompensas por episodio
plt.plot(episode_rewards)
plt.xlabel('Episodio')
plt.ylabel('Recompensa')
plt.title('Recompensa por Episodio durante la Evaluación')

# Guardar la gráfica en un archivo de imagen (por ejemplo, en formato PNG)
plt.savefig('./results/recompensas_por_episodio_evaluacion_v2_1.png')

# Mostrar la gráfica en la ventana
plt.show()

---------------------------------------
Fichero de los modelos entrenados: TD3_HumanoidBulletEnv-v0_0_v2_1
---------------------------------------
---------------------------------------
Configuración: TD3_HumanoidBulletEnv-v0_0_v2_1
---------------------------------------
